Import necessary libraries

In [1]:
from transformers import pipeline
import pandas as pd
import re
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, accuracy_score, precision_score, recall_score, f1_score

Upload zero-shot model. You can download it here https://huggingface.co/MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7.

In [ ]:
classifier = pipeline("zero-shot-classification", model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli")

Upload test dataset.We explained how to generate in dataset_generation file, here you can find a link to this file https://github.com/preste-ai/rnd-nlp-cot-chatgptdatagen/blob/main/dataset_generation.ipynb

In [3]:
dataset = pd.read_csv("Your test set")

Run zero-shot model on our data

In [18]:
candidate_labels = ["réservation_de_restaurant", "réserver_un_vol", "annuler_la_réservation", "état_du_vol", "réserver_un_hôtel", "suggestion_de_voyage", "visa_international", "transfert", "location_de_voiture", "bagages_perdus"]
def run_zero_shot(dataset, candidate_labels):
    data = dataset["text-fr"].tolist()
    pred_dict = []
    for sent in data:
        sequence_to_classify = sent
        output = classifier(sequence_to_classify, candidate_labels, multi_label=False)
        pred_dict.append(output)
    print(pred_dict)
    pred_labels = []
    for pred in pred_dict:
        pred_labels.append(pred["labels"][0])
    dataset['Predictions'] = pred_labels
    dataset['Predictions_encoding'] = dataset['Predictions'].replace({"réservation_de_restaurant" : 0 , "bagages_perdus" : 1,
                                     "annuler_la_réservation" : 2, "suggestion_de_voyage" : 3, 
                                     "visa_international" : 4, "état_du_vol" : 5, "transfert" : 6,
                                     "réserver_un_vol" : 7, "location_de_voiture" : 8, "réserver_un_hôtel" : 9
                                     })
    return dataset

In [32]:
dataset = run_zero_shot(dataset, candidate_labels)

Get metrics

In [24]:
def get_metrics(y_pred, y_test): 
    test_report = classification_report(y_test, y_pred, output_dict=True)
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')

    return test_report, accuracy, precision, recall, f1


In [26]:
test_report, accuracy, precision, recall, f1 = get_metrics(dataset["Predictions_encoding"], dataset["labels"])

/home/preste-dakam/Documents/Projects_Preste/MadMonkeys/Mad_Monkeys/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/preste-dakam/Documents/Projects_Preste/MadMonkeys/Mad_Monkeys/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/preste-dakam/Documents/Projects_Preste/MadMonkeys/Mad_Monkeys/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_di

In [29]:
print("Accuracy: ", accuracy)
print("Precision: ", precision)
print("Recall: ", recall)
print("F1: ", f1)

Accuracy:  0.7
Precision:  0.7
Recall:  0.7
F1:  0.7
